<a href="https://colab.research.google.com/github/EnisBerk/speech_audio_understanding/blob/master/tensorflow2pytorch_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip install numpy scipy resampy six pysoundfile pydub

!pip install -U git+https://github.com/Microsoft/MMdnn.git@master

!pip install tensorflow six


  Cloning https://github.com/Microsoft/MMdnn.git (to revision master) to /tmp/pip-req-build-nn79nnrd
  Stored in directory: /tmp/pip-ephem-wheel-cache-coucwdfo/wheels/8c/32/7c/0d1fe129de14a44724cbba5de825891bab6992851fe52606c5
Successfully built mmdnn
  Found existing installation: mmdnn 0.2.3
    Uninstalling mmdnn-0.2.3:
      Successfully uninstalled mmdnn-0.2.3


In [0]:
import tensorflow as tf

import torch
import imp
import numpy as np

from google.colab import files

from pydub import AudioSegment

In [3]:
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz
!git clone https://github.com/tensorflow/models.git
 # Copy the source files to the current directory.

!cp models/research/audioset/* .




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0   141M      0  0:00:01  0:00:01 --:--:--  141M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0  3961k      0 --:--:-- --:--:-- --:--:-- 3961k
fatal: destination path 'models' already exists and is not an empty directory.


In [4]:
import vggish_slim
import vggish_params
import vggish_input


def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
  """Define VGGish model, load the checkpoint, and save model.
  """
  vggish_slim.define_vggish_slim()
  checkpoint_path = 'vggish_model.ckpt'
  vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  
  vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)

  features_tensor = sess.graph.get_tensor_by_name(
      vggish_params.INPUT_TENSOR_NAME)
  embedding_tensor = sess.graph.get_tensor_by_name(
      vggish_params.OUTPUT_TENSOR_NAME)
  saver = tf.train.Saver()
  save_path = saver.save(sess, "tf_vggish_model.ckpt")
  print("INPUT_TENSOR_NAME: ",vggish_params.INPUT_TENSOR_NAME)
  print("OUTPUT_TENSOR_NAME: ",vggish_params.OUTPUT_TENSOR_NAME)
  layers = {'conv1': 'vggish/conv1/Relu',
            'pool1': 'vggish/pool1/MaxPool',
            'conv2': 'vggish/conv2/Relu',
            'pool2': 'vggish/pool2/MaxPool',
            'conv3': 'vggish/conv3/conv3_2/Relu',
            'pool3': 'vggish/pool3/MaxPool',
            'conv4': 'vggish/conv4/conv4_2/Relu',
            'pool4': 'vggish/pool4/MaxPool',
            'fc1': 'vggish/fc1/fc1_2/Relu',
            'fc2': 'vggish/fc2/Relu',
            'embedding': 'vggish/embedding',
            'features': 'vggish/input_features',
         }
  g = tf.get_default_graph()
  for k in layers:
    layers[k] = g.get_tensor_by_name( layers[k] + ':0')
    
  return {'features': features_tensor,
          'embedding': embedding_tensor,
          'layers': layers,
         }



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
tf.reset_default_graph()
sess = tf.Session()

vgg=CreateVGGishNetwork()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from vggish_model.ckpt
INPUT_TENSOR_NAME:  vggish/input_features:0
OUTPUT_TENSOR_NAME:  vggish/embedding:0


In [6]:
!mmconvert -sf tensorflow -in tf_vggish_model.ckpt.meta -iw tf_vggish_model.ckpt --dstNode vggish/embedding -df pytorch -om tf_to_pytorch_vggish_model.pth

Parse file [tf_vggish_model.ckpt.meta] with binary format successfully.
Tensorflow model file [tf_vggish_model.ckpt.meta] loaded successfully.
Tensorflow checkpoint file [tf_vggish_model.ckpt] loaded successfully. [18] variables loaded.
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
2019-02-25 02:26:33.363435: I tensorflow/tools/graph_transforms/transform_graph.cc:317] Applying fold_constants
2019-02-25 02:26:33.394145: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-02-25 02:26:33.394428: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x35851e0 executing computations on platform Host. Devices:
2019-02-25 02:26:33.394479: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-02-25 02:26:33.462310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one 

In [7]:
MainModel = imp.load_source('MainModel', "tf_to_pytorch_vggish_model.py")

the_model = torch.load("tf_to_pytorch_vggish_model.pth")
the_model.eval()


KitModel(
  (vggish_conv1_Conv2D): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (vggish_conv2_Conv2D): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (vggish_conv3_conv3_1_Conv2D): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (vggish_conv3_conv3_2_Conv2D): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (vggish_conv4_conv4_1_Conv2D): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (vggish_conv4_conv4_2_Conv2D): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (vggish_fc1_fc1_1_MatMul): Linear(in_features=12288, out_features=4096, bias=True)
  (vggish_fc1_fc1_2_MatMul): Linear(in_features=4096, out_features=4096, bias=True)
  (vggish_fc2_MatMul): Linear(in_features=4096, out_features=128, bias=True)
)

In [0]:
#Generate Input
num_secs = 3
freq = 1000
sr = 44100
t = np.linspace(0, num_secs, int(num_secs * sr))
x = np.sin(2 * np.pi * freq * t)  # Unit amplitude input signal




In [9]:
#THIS CELL WILL GIVE ERROR:

# line 39 at tf_to_pytorch_vggish_model.py changes input shape:
#  vggish_Reshape  = torch.reshape(input = x, shape = (-1,96,64,1))
# however it should be (-1,1,96,64) since pytorch uses N,C,H,W 

#pre-process input
input_batch = vggish_input.waveform_to_examples(x, sr)

input_batch=torch.from_numpy(input_batch)

output=the_model(input_batch)



RuntimeError: ignored

In [0]:
# GET MODIFIED tf_to_pytorch_vggish_model.py for correct input shape
!curl -O https://gist.githubusercontent.com/EnisBerk/321dcd19bf388eed8a0f5af5562324a3/raw/f581e7547c7eb6fa4cc46395d8370fddd1bbaa8f/tf_to_pytorch_vggish_model.py
# RELOAD It
MainModel = imp.load_source('MainModel', "tf_to_pytorch_vggish_model.py")

the_model = torch.load("tf_to_pytorch_vggish_model.pth")
the_model.eval()



In [0]:
#THIS CELL WILL GIVE ERROR:
#pytorch weights are float32 so input needs to be float32 as well
#pre-process input

input_batch = vggish_input.waveform_to_examples(x, sr)

input_batch=torch.from_numpy(input_batch)

output=the_model.forward(input_batch)


In [0]:
# Finally this works, however results are different than tensorflow
input_batch = vggish_input.waveform_to_examples(x, sr)
# make input float32
input_batch=input_batch.astype(np.float32)

input_batch=torch.from_numpy(input_batch)

output=the_model.forward(input_batch)


In [0]:
# INFERENCE WITH tensorflow
# Produce a batch of log mel spectrogram examples.
input_batch = vggish_input.waveform_to_examples(x, sr)

# cast to float to make sure inputs are same
input_batch=input_batch.astype(np.float32)

[embedding_batch] = sess.run([vgg['embedding']],
                             feed_dict={vgg['features']: input_batch})



In [0]:
print("checksum(tf): ",embedding_batch.sum())
print("checksum(pytorch):",output.sum())